In [1]:
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc, Input, Output
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime, timedelta

In [2]:
sector_symbols = { "IT": ["AAPL", "GOOGL", "NVDA"], "Finance": ["JPM", "C", "BLK"] }

In [3]:
#end_date = datetime.today()
end_date = datetime(2025, 6, 26)
start_date = end_date - timedelta(days=30)

# Dash app initialization
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [4]:
# Layout
app.layout = dbc.Container([
    html.H1("Stock Transaction Volume Dashboard", className="text-center my-4"),

    dbc.Row([
        dbc.Col([
            html.Label("Select Sector:"),
            dcc.Dropdown(
                id="sector-dropdown",
                options=[{"label": k, "value": k} for k in sector_symbols],
                value="IT",
                clearable=False
            )
        ], width=4),

        dbc.Col([
            html.Label("Select End Date:"),
            dcc.DatePickerSingle(
                id="date-picker",
                min_date_allowed=datetime(2020, 1, 1),
                max_date_allowed=end_date,
                date=end_date.date()
            )
        ], width=4)
    ], className="mb-4"),

    dbc.Row([
        dbc.Col([
            dcc.Loading(
                id="loading-graph",
                children=[dcc.Graph(id="volume-chart")],
                type="default"
            )
        ])
    ])
])


# dbc.Container: Bootstrap container
# html.H1: App title
# dcc.Dropdown: For selecting sector (id="sector-dropdown")
# dcc.DatePickerSingle: For selecting end date (id="date-picker")
# dcc.Graph: Area where the Plotly chart is rendered
# dcc.Loading: Shows a spinner while the graph is loading

In [5]:
# Callback for updating graph
@app.callback(
    Output("volume-chart", "figure"),
    Input("sector-dropdown", "value"),
    Input("date-picker", "date")
)

def update_graph(selected_sector, selected_date):
    tickers = sector_symbols[selected_sector]
    end = pd.to_datetime(selected_date)
    start = end - pd.Timedelta(days=30)

    df_dict = yf.download(tickers, start=start, end=end)["Volume"]

    if isinstance(df_dict, pd.Series):  # Only one ticker case
        df_dict = df_dict.to_frame()

    fig = go.Figure()
    for ticker in tickers:
        fig.add_trace(go.Scatter(
            x=df_dict.index,
            y=df_dict[ticker],
            mode="lines",
            name=ticker
        ))

    fig.update_layout(
        title=f"30-Day Daily Transaction Volume ({selected_sector} Sector)",
        xaxis_title="Date",
        yaxis_title="Volume",
        template="plotly_white"
    )

    return fig

In [6]:
app.run(debug=True)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed


## To allow the duration (currently hardcoded as 30 days) to be dynamically adjustable by the user, you can add a slider or number input to your Dash layout. Here's how to implement that:

## Steps to Add a Duration Selector:
### 1 Add a dcc.Input or dcc.Slider component to the layout to let the user choose the number of days.
### 2 Include it as an Input in your callback.
### 3 Use the selected value to compute the start_date.

In [7]:
sector_symbols = { "IT": ["AAPL", "GOOGL", "NVDA"], "Finance": ["JPM", "C", "BLK"] }
#end_date = datetime.today()
end_date = datetime(2025, 6, 26)
start_date = end_date - timedelta(days=252)

# Dash app initialization
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


In [8]:

# Layout
app.layout = dbc.Container([
    html.H1("Stock Transaction Volume Dashboard", className="text-center my-4"),

    dbc.Row([
        dbc.Col([
            html.Label("Select Sector:"),
            dcc.Dropdown(
                id="sector-dropdown",
                options=[{"label": k, "value": k} for k in sector_symbols],
                value="IT",
                clearable=False
            )
        ], width=4),

        dbc.Col([
            html.Label("Select End Date:"),
            dcc.DatePickerSingle(
                id="date-picker",
                min_date_allowed=datetime(2020, 1, 1),
                max_date_allowed=end_date,
                date=end_date.date()
            )
        ], width=4),

        dbc.Col([
            html.Label("Select Duration (days):"),
            dcc.Input(
                id="duration-input",
                type="number",
                min=1,
                max=365,
                step=1,
                value=252
            )
        ], width=4)
    ], className="mb-4"),


    dbc.Row([
        dbc.Col([
            dcc.Loading(
                id="loading-graph",
                children=[dcc.Graph(id="volume-chart")],
                type="default"
            )
        ])
    ])
])


# dbc.Container: Bootstrap container
# html.H1: App title
# dcc.Dropdown: For selecting sector (id="sector-dropdown")
# dcc.DatePickerSingle: For selecting end date (id="date-picker")
# dcc.Graph: Area where the Plotly chart is rendered
# dcc.Loading: Shows a spinner while the graph is loading
# Callback for updating graph

In [9]:

@app.callback(
    Output("volume-chart", "figure"),
    Input("sector-dropdown", "value"),
    Input("date-picker", "date"),
    Input("duration-input", "value")
)

def update_graph(selected_sector, selected_date, duration_days):
    tickers = sector_symbols[selected_sector]
    end = pd.to_datetime(selected_date)
    start = end - pd.Timedelta(days=duration_days)

    df_dict = yf.download(tickers, start=start, end=end)["Volume"]

    if isinstance(df_dict, pd.Series):  # Only one ticker case
        df_dict = df_dict.to_frame()

    fig = go.Figure()
    for ticker in tickers:
        fig.add_trace(go.Scatter(
            x=df_dict.index,
            y=df_dict[ticker],
            mode="lines",
            name=ticker
        ))

    fig.update_layout(
        title=f"30-Day Daily Transaction Volume ({selected_sector} Sector)",
        xaxis_title="Date",
        yaxis_title="Volume",
        template="plotly_white"
    )

    return fig


In [ ]:
app.run(debug=True)

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed


# Done for the FX_Swap dataset now, with ticker names, different tenors, volumes and dates

In [14]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Parameters
tickers = ['EURUSD', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCAD']
buckets = ['ON', '1W', '1M', '3M', '6M']  # Reduced to 5 tenors
num_days = 40  # Generate data for the last 30 days

# Generate daily dates
base_date = datetime.today().date()
dates = [base_date - timedelta(days=i) for i in range(num_days)]

# Generate data
data = []
for ticker in tickers:
    for date in dates:
        for bucket in buckets:
            volume = round(random.uniform(1e6, 1e8), 2)  # Random volume
            data.append({
                'ticker': ticker,
                'bucket': bucket,
                'volume': volume,
                'date': date
            })

# Create DataFrame
FX_Swap_df = pd.DataFrame(data)

# Display sample
FX_Swap_df


,ticker,bucket,volume,date
0,EURUSD,ON,85502592.55,2025-07-01
1,EURUSD,1W,27984225.66,2025-07-01
2,EURUSD,1M,22503242.79,2025-07-01
3,EURUSD,3M,4458445.45,2025-07-01
4,EURUSD,6M,29394129.02,2025-07-01
...,...,...,...,...
995,USDCAD,ON,9829216.49,2025-05-23
996,USDCAD,1W,2875001.97,2025-05-23
997,USDCAD,1M,11641754.06,2025-05-23
998,USDCAD,3M,74748733.13,2025-05-23


# Deepseek


In [21]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Sample data creation (if FX_Swap_df doesn't exist)
try:
    FX_Swap_df
except NameError:
    np.random.seed(42)
    tickers = ['EURUSD', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCAD', 'NZDUSD']
    buckets = ['ON', '1W', '1M', '3M', '6M', '1Y', '2Y']
    dates = pd.date_range(end=datetime.today(), periods=90).date
    
    data = []
    for date in dates:
        for ticker in tickers:
            for bucket in buckets:
                # Create some trends in the data
                base_vol = np.random.randint(100, 500)
                trend = (date - dates[0]).days / 10
                seasonal = np.sin(2 * np.pi * (date - dates[0]).days / 30) * 50
                data.append({
                    'ticker': ticker,
                    'bucket': bucket,
                    'volume': int(base_vol + trend + seasonal + np.random.normal(0, 20)),
                    'date': date
                })
    
    FX_Swap_df = pd.DataFrame(data)

# Convert date column to datetime
FX_Swap_df['date'] = pd.to_datetime(FX_Swap_df['date'])
max_date = FX_Swap_df['date'].max()

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server

# Get unique tickers and buckets for dropdowns
all_tickers = sorted(FX_Swap_df['ticker'].unique())
all_buckets = sorted(FX_Swap_df['bucket'].unique())

app.layout = html.Div([
    html.Div([
        html.H1("FX Swap Volume Analysis Dashboard", style={'textAlign': 'center'}),
        
        html.Div([
            html.Div([
                html.Label("Select Tickers:", style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='ticker-selector',
                    options=[{'label': t, 'value': t} for t in all_tickers],
                    value=all_tickers[:3],  # Default to first 3 tickers
                    multi=True,
                    style={'width': '100%'}
                ),
            ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
            
            html.Div([
                html.Label("Time Window (days from most recent date):", style={'fontWeight': 'bold'}),
                dcc.Slider(
                    id='time-window-slider',
                    min=1,
                    max=180,
                    value=30,
                    marks={i: str(i) for i in range(0, 181, 30)},
                    step=1,
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ], style={'width': '65%', 'display': 'inline-block', 'padding': '10px'}),
            
            html.Div([
                html.Label("Additional Options:", style={'fontWeight': 'bold'}),
                dcc.Checklist(
                    id='display-options',
                    options=[
                        {'label': ' Show Trend Lines', 'value': 'show_trend'},
                        {'label': ' Normalize Scales', 'value': 'normalize'},
                        {'label': ' Show Data Table', 'value': 'show_table'}
                    ],
                    value=[],
                    labelStyle={'display': 'inline-block', 'marginRight': '20px'}
                )
            ], style={'width': '100%', 'padding': '10px'}),
        ], style={'border': '1px solid #ddd', 'borderRadius': '5px', 'padding': '10px', 'margin': '10px 0'}),
        
        dcc.Graph(id='volume-matrix-plot'),
        
        html.Div(id='data-table-container', style={'margin': '20px 0'})
    ], style={'padding': '20px'})
])

@app.callback(
    [Output('volume-matrix-plot', 'figure'),
     Output('data-table-container', 'children')],
    [Input('ticker-selector', 'value'),
     Input('time-window-slider', 'value'),
     Input('display-options', 'value')]
)
def update_plots(selected_tickers, time_window_days, display_options):
    # Filter data based on selections
    cutoff_date = max_date - timedelta(days=time_window_days)
    filtered_df = FX_Swap_df[
        (FX_Swap_df['date'] >= cutoff_date) & 
        (FX_Swap_df['ticker'].isin(selected_tickers))
    ].sort_values(['ticker', 'bucket', 'date'])
    
    # Get unique buckets from filtered data
    buckets = sorted(filtered_df['bucket'].unique())
    
    # Create subplots matrix
    fig = make_subplots(
        rows=len(buckets), 
        cols=len(selected_tickers),
        subplot_titles=selected_tickers,  # Column titles
        shared_yaxes='normalize' in display_options,
        vertical_spacing=0.05,
        horizontal_spacing=0.05
    )
    
    # Add row labels (buckets)
    for i, bucket in enumerate(buckets):
        fig.add_annotation(
            x=-0.07,
            y=1 - (i + 0.5)/len(buckets),
            xref="paper",
            yref="paper",
            text=bucket,
            showarrow=False,
            textangle=-90,
            font=dict(size=12, color='black'),
            bgcolor='rgba(255,255,255,0.7)'
        )
    
    # Populate each subplot
    for col_idx, ticker in enumerate(selected_tickers, start=1):
        for row_idx, bucket in enumerate(buckets, start=1):
            subset = filtered_df[
                (filtered_df['ticker'] == ticker) & 
                (filtered_df['bucket'] == bucket)
            ]
            
            # Add bar chart
            fig.add_trace(
                go.Bar(
                    x=subset['date'],
                    y=subset['volume'],
                    name=f"{ticker} {bucket}",
                    showlegend=False,
                    marker_color='#1f77b4',
                    opacity=0.7
                ),
                row=row_idx,
                col=col_idx
            )
            
            # Add trend line if selected
            if 'show_trend' in display_options and len(subset) > 1:
                fig.add_trace(
                    go.Scatter(
                        x=subset['date'],
                        y=subset['volume'].rolling(7, min_periods=1).mean(),
                        mode='lines',
                        line=dict(color='red', width=2),
                        name='Trend',
                        showlegend=False
                    ),
                    row=row_idx,
                    col=col_idx
                )
            
            # Update axes
            fig.update_xaxes(
                title_text="Date" if row_idx == len(buckets) else "",
                row=row_idx,
                col=col_idx,
                tickformat="%b %d",
                tickangle=45
            )
            
            if col_idx == 1:
                fig.update_yaxes(
                    title_text="Volume",
                    row=row_idx,
                    col=col_idx
                )
    
    # Update layout
    fig.update_layout(
        height=200 * len(buckets),
        width=300 * max(len(selected_tickers), 1),
        title_text=f"FX Swap Volume Evolution (Last {time_window_days} Days)",
        margin=dict(l=100, r=50, b=100, t=50),
        hovermode="closest",
        plot_bgcolor='rgba(240,240,240,0.5)'
    )
    
    # Create data table if selected
    table = None
    if 'show_table' in display_options and len(filtered_df) > 0:
        summary_df = filtered_df.groupby(['ticker', 'bucket'])['volume'].agg(['mean', 'min', 'max', 'std'])
        summary_df = summary_df.reset_index().round(2)
        
        table = dash_table.DataTable(
            id='data-table',
            columns=[{"name": i, "id": i} for i in summary_df.columns],
            data=summary_df.to_dict('records'),
            style_table={'overflowX': 'auto'},
            style_cell={
                'minWidth': '100px', 'width': '150px', 'maxWidth': '200px',
                'whiteSpace': 'normal'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        )
    
    return fig, [html.H3("Summary Statistics"), table] if table else None

if __name__ == '__main__':
    app.run(debug=True)

# GPT

In [20]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Sample dataframe placeholder (replace this with your actual FX_Swap_df)
# FX_Swap_df = pd.read_csv("your_data.csv")
# Example format:
# FX_Swap_df = pd.DataFrame({
#     'ticker': ['EURUSD', 'EURUSD', 'USDJPY', 'USDJPY'],
#     'bucket': ['1W', '1M', '1W', '1M'],
#     'volume': [100, 150, 200, 250],
#     'date': pd.to_datetime(['2023-06-01', '2023-06-02', '2023-06-01', '2023-06-02'])
# })

# Create the Dash app
app = dash.Dash(__name__)
server = app.server  # For deployment

# Assume FX_Swap_df is already defined
def create_app_layout(df):
    max_date = df['date'].max()
    min_date = df['date'].min()
    default_days = 30

    return html.Div([
        html.H2("FX Swap Volume Explorer"),
        
        html.Label("Select time window (days from most recent date):"),
        dcc.Slider(
            id='days-slider',
            min=1,
            max=(max_date - min_date).days,
            value=default_days,
            marks={i: str(i) for i in range(0, (max_date - min_date).days + 1, 10)},
            step=1
        ),

        html.Br(),

        html.Label("Select Tickers:"),
        dcc.Dropdown(
            id='ticker-dropdown',
            options=[{'label': t, 'value': t} for t in sorted(df['ticker'].unique())],
            value=sorted(df['ticker'].unique()),
            multi=True
        ),

        html.Label("Select Buckets (Tenors):"),
        dcc.Dropdown(
            id='bucket-dropdown',
            options=[{'label': b, 'value': b} for b in sorted(df['bucket'].unique())],
            value=sorted(df['bucket'].unique()),
            multi=True
        ),

        html.Br(),
        html.Div(id='plots-container')
    ])


@app.callback(
    Output('plots-container', 'children'),
    Input('days-slider', 'value'),
    Input('ticker-dropdown', 'value'),
    Input('bucket-dropdown', 'value')
)
def update_plots(days, selected_tickers, selected_buckets):
    recent_date = FX_Swap_df['date'].max()
    cutoff_date = recent_date - pd.Timedelta(days=days)

    filtered_df = FX_Swap_df[
        (FX_Swap_df['date'] >= cutoff_date) &
        (FX_Swap_df['ticker'].isin(selected_tickers)) &
        (FX_Swap_df['bucket'].isin(selected_buckets))
    ]

    plots_grid = []
    for bucket in sorted(selected_buckets):
        row = []
        for ticker in sorted(selected_tickers):
            sub_df = filtered_df[(filtered_df['ticker'] == ticker) & (filtered_df['bucket'] == bucket)]
            if sub_df.empty:
                fig = go.Figure()
                fig.update_layout(title=f"{ticker} - {bucket}", height=200, margin=dict(l=10, r=10, t=30, b=10))
            else:
                fig = px.bar(sub_df, x='date', y='volume', title=f"{ticker} - {bucket}")
                fig.update_layout(height=200, margin=dict(l=10, r=10, t=30, b=10))

            row.append(html.Div(dcc.Graph(figure=fig), style={'width': '300px', 'display': 'inline-block'}))
        plots_grid.append(html.Div(row, style={'whiteSpace': 'nowrap'}))

    return plots_grid


# Run the app
if __name__ == '__main__':
    # Make sure to load FX_Swap_df here or define it earlier
    app.layout = create_app_layout(FX_Swap_df)
    app.run(debug=True)
